In [1]:
import numpy as np

In [2]:
with open(r"Deep Learning.txt") as data:
    train_data = data.read()

In [3]:
def preprocess(data):
    data = data.replace('\n',' ')
    chars = list(set(data))
    char_to_num = {char:idx for idx,char in enumerate(chars)}
    num_to_char = {idx:char for idx,char in enumerate(chars)}
    return data,chars,char_to_num , num_to_char

In [4]:
def generate_training_data(train_data,seq):
    input_seqs = []
    target_seqs = []
    for index in range(len(train_data)-seq):
        input_seqs.append([char_to_num[char] for char in train_data[index:index+seq]])
        target_seqs.append(char_to_num[train_data[index+seq]])
    return np.array(input_seqs) , np.array(target_seqs)

In [5]:
train_data , unique_char , char_to_num , num_to_char = preprocess(train_data)

In [6]:
max_sequence_length = 100
x , y = generate_training_data(train_data,max_sequence_length)

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense , Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
vocab_size = len(unique_char)
embedding_dim = 256
hidden_dim = 128
max_sequence_length = 100

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length),
    LSTM(units=hidden_dim, return_sequences=True),
    LSTM(units=hidden_dim, return_sequences=False),
    Dense(units=hidden_dim, activation='relu'),
    Dense(units=vocab_size, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 256)          13312     
                                                                 
 lstm (LSTM)                 (None, 100, 128)          197120    
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dense_1 (Dense)             (None, 52)                6708      
                                                                 
Total params: 365236 (1.39 MB)
Trainable params: 365236 (1.39 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
x.shape , y.shape

((18601, 100), (18601,))

In [10]:
history = model.fit(x,y, epochs=75 , batch_size = 64 )

Epoch 1/75
291/291 [==============================] - 20s 30ms/step - loss: 2.9622 - accuracy: 0.1678
Epoch 2/75
291/291 [==============================] - 4s 14ms/step - loss: 2.4559 - accuracy: 0.2983
Epoch 3/75
291/291 [==============================] - 3s 12ms/step - loss: 2.1525 - accuracy: 0.3689
Epoch 4/75
291/291 [==============================] - 4s 12ms/step - loss: 1.9546 - accuracy: 0.4251
Epoch 5/75
291/291 [==============================] - 4s 13ms/step - loss: 1.7894 - accuracy: 0.4744
Epoch 6/75
291/291 [==============================] - 3s 11ms/step - loss: 1.6525 - accuracy: 0.5127
Epoch 7/75
291/291 [==============================] - 4s 12ms/step - loss: 1.5417 - accuracy: 0.5443
Epoch 8/75
291/291 [==============================] - 4s 14ms/step - loss: 1.4355 - accuracy: 0.5765
Epoch 9/75
291/291 [==============================] - 4s 12ms/step - loss: 1.3467 - accuracy: 0.5991
Epoch 10/75
291/291 [==============================] - 3s 12ms/step - loss: 1.2602 - accur

In [12]:
seq_length = 100
sentence_end_markers = ['.', '?']

def generate_text(model, start_text, char_to_index, index_to_char, max_length=500):
    generated_text = start_text
    num_words = len(generated_text.split())

    while num_words < max_length and not any(generated_text.endswith(marker) for marker in sentence_end_markers):
        input_seq = [char_to_index[char] for char in generated_text[-seq_length:]]
        input_seq = pad_sequences([input_seq], maxlen=seq_length, padding='pre')

        probabilities = model.predict(input_seq, verbose=0)[0]
        predicted_index = np.random.choice(len(probabilities), p=probabilities)
        predicted_char = index_to_char[predicted_index]

        generated_text += predicted_char
        num_words = len(generated_text.split())

    return generated_text


In [13]:
start_seq = "Deep Learning is a subset of Machine Learning"
generated_text = generate_text(model, start_seq, char_to_num, num_to_char, max_length=500)

print("Generated Text:")
print(generated_text)

Generated Text:
Deep Learning is a subset of Machine Learning and enhance and algorithms, share exploring techniques to push the boundaries of AI and unbiased impact on intelligent drug experiences, Deep Learning is a rapidly evolution.
